In [1]:
import os
import re
import numpy as np
import pydicom
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px


def load_dicom_series(dicom_dir: str) -> np.ndarray:
    dicom_files = sorted([os.path.join(dicom_dir, f) for f in os.listdir(dicom_dir) if f.endswith('.DCM')])
    slices = [pydicom.dcmread(dcm) for dcm in dicom_files]
    slices.sort(key=lambda x: int(x.InstanceNumber)) 
    volume = np.stack([s.pixel_array for s in slices], axis=-1)
    return volume

def extract_lambda_value(path: str) -> str:
    match = re.search(r"recon_(\d+(\.\d+)?)", path)
    return match.group(1) if match else "Unknown"

DICOM_DIRS = [
    "/Users/akshdeepsandhu/Desktop/MRI/imrh_114a/imoco_recon_0",
    "/Users/akshdeepsandhu/Desktop/MRI/imrh_114a/imoco_recon_0.01",
    "/Users/akshdeepsandhu/Desktop/MRI/imrh_114a/imoco_recon_0.025",
    "/Users/akshdeepsandhu/Desktop/MRI/imrh_114a/imoco_recon_0.05",
    "/Users/akshdeepsandhu/Desktop/MRI/imrh_114a/imoco_recon_0.075",
]


volumes = [load_dicom_series(d) for d in DICOM_DIRS]
labels = [extract_lambda_value(d) for d in DICOM_DIRS]
dimensions = volumes[0].shape  

In [2]:
# Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("DICOM Viewer - Multiple Volumes"),
    
    html.Div([
        html.Label("Select Axis:"),
        dcc.Dropdown(
            id='axis-selector',
            options=[
                {'label': 'X-Axis', 'value': 'x'},
                {'label': 'Y-Axis', 'value': 'y'},
                {'label': 'Z-Axis', 'value': 'z'}
            ],
            value='z'
        )
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Label("Slice Index:"),
        dcc.Slider(
            id='slice-slider',
            min=0,
            max=dimensions[2] - 1,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, dimensions[2], max(1, dimensions[2] // 10))}
        )
    ], style={'width': '60%', 'display': 'inline-block', 'padding': '0px 20px'}),
    
    html.Div([
        html.Label("Adjust Contrast:"),
        dcc.Slider(
            id='contrast-slider',
            min=0,
            max=1,
            step=0.01,
            value=1.0,
            marks={i: f"{i}" for i in range(0, 2)}
        )
    ], style={'width': '30%', 'display': 'inline-block'}),
    
    html.Div([
        html.Div(id=f'graph-container-{i}', style={'width': '19%', 'display': 'inline-block', 'margin': '0 0.5%'})
        for i in range(5)
    ], style={'display': 'flex', 'justify-content': 'space-between'})
])

@app.callback(
    [Output(f'graph-container-{i}', 'children') for i in range(5)],
    [Input('axis-selector', 'value'),
     Input('slice-slider', 'value'),
     Input('contrast-slider', 'value')]
)
def update_slices(axis, index, contrast):
    figs = []
    for i, volume in enumerate(volumes):
        if axis == 'x':
            slice_2d = volume[index, :, :]
            title = f'Slice at X={index} (Lambda: {labels[i]})'
        elif axis == 'y':
            slice_2d = volume[:, index, :]
            title = f'Slice at Y={index} (Lambda: {labels[i]})'
        elif axis == 'z':
            slice_2d = volume[:, :, index]
            title = f'Slice at Z={index} (Lambda: {labels[i]})'
        else:
            slice_2d = np.zeros_like(volume[:, :, 0]) 
            title = "Invalid Axis"

       # contrast
        slice_2d = np.clip(slice_2d * contrast, 0, 255)  
        # rotate
        slice_2d = np.rot90(slice_2d, k=3)

        # plot
        fig = px.imshow(slice_2d, color_continuous_scale='gray')
        fig.update_layout(
            title=title,
            coloraxis_showscale=False,
            margin=dict(l=10, r=10, t=40, b=10) 
        )

        figs.append(dcc.Graph(figure=fig, style={'height': '400px'}))  # Smaller height to fit multiple graphs

    return figs

if __name__ == '__main__':
    app.run_server(debug=True)
